# Réseau de neurones avec TensorFlow

In [14]:
# Charger les packages nécessaires

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [15]:
print(tf.__version__)

2.4.0


In [16]:
# Importer le jeu de données

mnist = fetch_openml(name="mnist_784", version=1)

In [18]:
# Séparer les features et la target

X, y = mnist['data'].values, mnist['target'].values


X = X.reshape(-1, 28, 28, 1)
y = y.astype(np.uint8)
X.shape

(70000, 28, 28, 1)

In [19]:
# Séparer le jeu de données en jeu d'apprentissage et de test non aléatoire avec 10000 valeurs de test

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=1/7)

In [20]:
# Normaliser le jeu de données pour un meilleur apprentissage (permet de gagner 7% d'accuracy)

X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

In [23]:
# Création du modèle avec 3 couches

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu',  input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Entrainement du modèle

model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 62s 33ms/step - loss: 0.2919 - accuracy: 0.9137 - val_loss: 0.0596 - val_accuracy: 0.9820
Epoch 2/5
1875/1875 [==============================] - 62s 33ms/step - loss: 0.0570 - accuracy: 0.9827 - val_loss: 0.0527 - val_accuracy: 0.9837
Epoch 3/5
1875/1875 [==============================] - 61s 33ms/step - loss: 0.0360 - accuracy: 0.9888 - val_loss: 0.0519 - val_accuracy: 0.9841
Epoch 4/5
1875/1875 [==============================] - 62s 33ms/step - loss: 0.0225 - accuracy: 0.9926 - val_loss: 0.0487 - val_accuracy: 0.9868
Epoch 5/5
1875/1875 [==============================] - 62s 33ms/step - loss: 0.0147 - accuracy: 0.9950 - val_loss: 0.0582 - val_accuracy: 0.9853


In [25]:
#Prédiction des 30 premières valeurs et les comparer avec les valeurs réelles

y_pred = model.predict(X_test[:30])

print("Réel: ", y_test[:30])
print("Pred: ",np.argmax(y_pred, axis=1))

Réel:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]
Pred:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1]


In [26]:
# Sauvegarder le modèle pour pouvoir le réutiliser dans l'interface graphique

model.save('mnist_model')

INFO:tensorflow:Assets written to: mnist_model\assets
